In [1]:
import sys
import os
sys.path.append(os.path.abspath("Pytorch/"))
sys.path.append(os.path.abspath("models/"))

In [2]:
from XVA import *
from XVAFBSNNs import *

In [3]:
from FBSNNs import *
from CallOptionsBasket import *

In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 100  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
model = CallOptionsBasket(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 2 * 10**4
lr = 1e-3

In [5]:
model.load_model("models/100DCallOption.pth")

In [6]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 100  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [1+1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
xvamodel = XVA(Xi, T, M, N, D, Mm, layers, mode, activation, model)

n_iter = 2 * 10**3
lr = 1e-3

In [8]:
n_iter = 1 * 10**3
lr = 1e-4

In [9]:
tot = time.time()
print(xvamodel.device)
graph = xvamodel.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 1900, Loss: 5.981e-04, Y0: -0.050, Time: 0.60, Learning Rate: 1.000e-04
It: 2000, Loss: 5.961e-04, Y0: -0.047, Time: 61.16, Learning Rate: 1.000e-04
It: 2100, Loss: 5.374e-04, Y0: -0.048, Time: 60.91, Learning Rate: 1.000e-04
It: 2200, Loss: 3.948e-04, Y0: -0.048, Time: 60.29, Learning Rate: 1.000e-04
It: 2300, Loss: 4.823e-04, Y0: -0.048, Time: 60.15, Learning Rate: 1.000e-04
It: 2400, Loss: 4.802e-04, Y0: -0.049, Time: 60.70, Learning Rate: 1.000e-04
It: 2500, Loss: 3.905e-04, Y0: -0.048, Time: 60.51, Learning Rate: 1.000e-04
It: 2600, Loss: 4.347e-04, Y0: -0.048, Time: 60.73, Learning Rate: 1.000e-04
It: 2700, Loss: 4.835e-04, Y0: -0.048, Time: 60.39, Learning Rate: 1.000e-04
It: 2800, Loss: 5.115e-04, Y0: -0.049, Time: 62.53, Learning Rate: 1.000e-04
total time: 607.7461543083191 s


In [10]:
t_test, W_test, C_test = xvamodel.fetch_minibatch()
X_pred, Y_pred = xvamodel.predict(C_test, t_test, W_test)

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

for i in range(31):
    t_test_i, W_test_i, C_test_i = xvamodel.fetch_minibatch()
    X_pred_i, Y_pred_i = xvamodel.predict(C_test_i, t_test_i, W_test_i)
    if type(X_pred_i).__module__ != 'numpy':
        X_pred_i = X_pred_i.cpu().detach().numpy()
    if type(Y_pred_i).__module__ != 'numpy':
        Y_pred_i = Y_pred_i.cpu().detach().numpy()
    X_pred = np.concatenate((X_pred, X_pred_i), axis=0)
    Y_pred = np.concatenate((Y_pred, Y_pred_i), axis=0)

In [23]:
Y_preds = Y_pred.reshape(2048, 101)

In [39]:
pred_fva = Y_preds[:,0][0]

In [31]:
from scipy.stats import multivariate_normal as normal
dw_sample = normal.rvs(size=[40000, D, N]) * np.sqrt(T/N)
x_sample = np.zeros([40000, D, N + 1]) 
x_sample[:, :, 0] = np.ones([40000, D]) * 1
factor = np.exp((0.05-(0.4**2)/2)*(T/N))
for i in range(N):   
    x_sample[:, :, i + 1] = (factor * np.exp(0.4 * dw_sample[:, :, i])) * x_sample[:, :, i]

In [32]:
portfolio = np.sum(x_sample, axis=1, keepdims=True)
payoff = np.maximum(portfolio - 100 * 1, 0)
average = np.mean(payoff, axis=0, keepdims=True)
mc_price = np.exp(-0.05 * 1) * average[:, :, 100]

In [33]:
mc_price[0][0]

5.09535087353754

In [34]:
mc_price_rf = np.exp(-0.04 * 1) * average[:, :, 100]

In [35]:
mc_price_rf[0][0]

5.146560001169053

In [40]:
exact_fva = mc_price[0][0] - mc_price_rf[0][0]

In [41]:
pred_fva

-0.048008636

In [42]:
exact_fva

-0.05120912763151342

In [43]:
(pred_fva - exact_fva) ** 2 / exact_fva ** 2

0.003906058421267306

In [37]:
xvamodel.save_model("models/100DCallOptionFVA.pth")